In [1]:
import pandas as pd
import math as mt
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import statistics
from itertools import islice

In [2]:
url = "distances_table_nextbike_2020-07.csv"
#url_pr = "processed_data_nextbike_2020-06cc.csv"
df = pd.read_csv(url, sep=';', low_memory=False)
#df_pr = pd.read_csv(url, sep=';', low_memory=False)

In [3]:
pd.set_option('display.max_rows()', None)

#### Function for limiting possible lengths of distances to 2500meters in order to work with one part of data without extreme values

In [4]:
def drop_duplicates(l):
    '''Droping duplicates from a list while maintaining order'''
    seen = set()
    seen_add = seen.add
    return [x for x in l if not (x in seen or seen_add(x))]

In [5]:
def short_list(time):
    return [x for x in sr[time] if x < 2500]

In [6]:
df = df.drop(['bike', 'bike_racks', 'bikes', 'booked_bikes', 'free_racks', 'free_special_racks', 'maintenance', 'number', 
              'place_type', 'rack_locks', 'special_racks', 'spot', 'terminal_type', 'city',
              'country_code', 'company', 'timezone'], axis=1)

In [7]:
df.drop(['index'], axis=1, inplace=True)

In [8]:
df.head()

,lat_begin,lng_begin,lat_end,lng_end,ride_begin,ride_end,ride_time_minutes,distance,uid_begin,uid_end,standing_before_ride,address,name,available_bikes
0,50.921618,6.933237,50.913049,6.922490,2020-07-01 09:31:02.322428,2020-07-01 09:49:01.995455,18.0,1215.0,31804315,31819860,571.0,NaN,BIKE 22933,1093
1,50.913049,6.922490,50.910794,6.941967,2020-07-01 10:35:02.336121,2020-07-01 10:44:02.333799,9.0,1389.0,31819860,31821838,46.0,NaN,BIKE 22933,1159
2,50.910794,6.941967,50.931395,6.918226,2020-07-01 11:32:02.897704,2020-07-01 11:55:01.902627,23.0,2832.0,31821838,31824497,48.0,NaN,BIKE 22933,1161
3,50.931395,6.918226,50.929816,6.913806,2020-07-01 12:03:01.601807,2020-07-01 12:24:02.273590,21.0,356.0,31824497,31825694,8.0,NaN,BIKE 22933,1166
4,50.929816,6.913806,50.929823,6.913615,2020-07-01 17:59:02.213975,2020-07-01 18:05:02.088433,6.0,13.0,31825694,31843292,335.0,NaN,BIKE 22933,1160


In [9]:
negative_values = df[df['ride_time_minutes'] < 0].index.to_list()
negative_values

[16937, 23609, 62091]

In [10]:
df.drop(negative_values, inplace=True)

In [11]:
rows_with_nan = []
for index, row in df.iterrows():
    is_nan_series = row.isnull()
    if is_nan_series.any():
        rows_with_nan.append(index)

In [12]:
df_adr = df.drop(rows_with_nan, axis=0)

In [13]:
df_adr

,lat_begin,lng_begin,lat_end,lng_end,ride_begin,ride_end,ride_time_minutes,distance,uid_begin,uid_end,standing_before_ride,address,name,available_bikes
165,50.942183,6.908841,50.935878,6.960586,2020-07-26 15:44:02.424175,2020-07-26 16:33:02.648095,49.0,3694.0,33123695,33182396,1330.0,Oskar-Jäger-Straße 117,BIKE 21941,1060
286,50.946200,6.956244,50.981890,6.944558,2020-07-28 00:58:02.744485,2020-07-28 01:20:02.389113,22.0,4053.0,33259931,33261789,68.0,Eigelstein 29 50668 Köln,BIKE 21926,1050
362,50.955875,6.905937,50.946823,6.939862,2020-07-12 23:31:02.183860,2020-07-13 02:10:02.854116,159.0,2582.0,32390824,32419767,483.0,Alpenerplatz,BIKE 22337,1048
436,50.953874,6.979436,50.938466,6.968394,2020-07-12 15:21:02.490744,2020-07-12 15:33:01.725518,12.0,1880.0,32376387,32392279,452.0,Köln - Hertastraße 1,BIKE 22246,1062
493,50.943225,6.932855,50.945990,6.923978,2020-07-28 18:44:02.536848,2020-07-28 18:53:02.999918,9.0,694.0,33297659,33302308,20.0,Ludolf-Camphausen-Straße / Venloerstraße,BIKE 21718,1085
749,50.909600,6.962694,50.933187,6.950228,2020-07-17 20:30:03.207563,2020-07-17 21:00:04.451363,30.0,2765.0,32589833,32690154,2232.0,"Bonner Str. 226, 50968 Köln",BIKE 21839,1029
808,50.968757,6.905158,50.980748,6.895676,2020-07-26 05:44:03.223903,2020-07-26 05:53:02.210996,9.0,1490.0,33119251,33165387,416.0,Rochusstraße 236,BIKE 21839,1058
920,50.946308,6.945868,50.936545,6.939375,2020-07-17 09:57:03.044231,2020-07-17 10:08:02.859811,11.0,1177.0,32602628,32655500,654.0,hansaring,BIKE 22473,990
1006,50.975220,6.965728,50.977784,6.968597,2020-07-22 20:55:02.436368,2020-07-22 21:36:02.739861,41.0,349.0,32948841,32973091,60.0,Eichhornstraße 2 50735 Köln,BIKE 22427,1019
1017,50.907458,6.972123,50.897884,6.964939,2020-07-27 17:32:03.000020,2020-07-29 21:32:02.701621,3120.0,1178.0,33238376,33372913,0.0,Krohstraße 4 50968 Köln,BIKE 22427,1075


In [14]:
address_list = df_adr.index.to_list()

In [15]:
previous_address_list = [(i-1) for i in address_list]
following_address_list = [(i+1) for i in address_list]

In [16]:
overview_list = previous_address_list + address_list + following_address_list
overview_list.sort()
overview_list = drop_duplicates(overview_list)

In [17]:
df_suspicious_indexes = df.loc[overview_list]

In [33]:
df_suspicious_indexes.head()

,lat_begin,lng_begin,lat_end,lng_end,ride_begin,ride_end,ride_time_minutes,distance,uid_begin,uid_end,standing_before_ride,address,name,available_bikes
164,50.965884,6.941094,50.942183,6.908841,2020-07-25 11:13:02.306074,2020-07-25 17:34:02.569958,381.0,3472.0,33015957,33123695,2516.0,NaN,BIKE 21941,1071
165,50.942183,6.908841,50.935878,6.960586,2020-07-26 15:44:02.424175,2020-07-26 16:33:02.648095,49.0,3694.0,33123695,33182396,1330.0,Oskar-Jäger-Straße 117,BIKE 21941,1060
166,50.935878,6.960586,50.912306,6.940547,2020-07-26 16:37:02.787397,2020-07-26 16:55:02.051694,18.0,2975.0,33182396,33184532,4.0,NaN,BIKE 21941,1062
285,50.953455,6.957361,50.946200,6.956244,2020-07-27 23:35:02.973704,2020-07-27 23:50:03.290941,15.0,811.0,33257006,33259931,73.0,NaN,BIKE 21926,1037
286,50.946200,6.956244,50.981890,6.944558,2020-07-28 00:58:02.744485,2020-07-28 01:20:02.389113,22.0,4053.0,33259931,33261789,68.0,Eigelstein 29 50668 Köln,BIKE 21926,1050


In [34]:
with open('mytable.tex','w') as tf:
    tf.write(df.head().to_latex())

In [19]:
previous_index = overview_list[0]
previous_distance = df_suspicious_indexes.iloc[0]['distance']
suspicious_return_trips = []
for index,row in islice(df_suspicious_indexes.iterrows(), 1, None):
    if (row['distance'] == previous_distance):
        suspicious_return_trips.append(previous_index)
        suspicious_return_trips.append(index)
        
    previous_index = index
    previous_distance = row['distance']

In [20]:
suspicious_return_trips

[1863, 1864, 9645, 9646, 9646, 9647]

In [21]:
df.loc[23173]

lat_begin                                  50.9431
lng_begin                                  6.95066
lat_end                                     50.737
lng_end                                    7.10255
ride_begin              2020-07-21 12:08:02.600019
ride_end                2020-07-21 15:09:02.525191
ride_time_minutes                              181
distance                                     25284
uid_begin                                 32870717
uid_end                                   32885221
standing_before_ride                           172
address                                        NaN
name                                    BIKE 21356
available_bikes                                995
Name: 23173, dtype: object

In [22]:
suspicious_distances = []
for index,row in df.iterrows():
    if (row['distance'] > 20000):
        suspicious_distances.append(index)

In [23]:
suspicious_distances

[1863,
 1864,
 7275,
 9712,
 9713,
 12857,
 12858,
 13962,
 13964,
 23173,
 23176,
 23511,
 23512,
 26549,
 26550,
 36634,
 36635,
 39716,
 39717,
 43582,
 43583,
 44096,
 44097,
 49139,
 49140,
 51948,
 51949,
 55589,
 55590]

In [24]:
#suspicious_times = []
#for index, row in df.iterrows():
    #if row['ride_time_minutes'] > 2880:
        #suspicious_times.append(index)

In [25]:
suspicious_times #remains to be investigated

NameError: name 'suspicious_times' is not defined

In [ ]:
single_bike1 = df[df['name']== 'BIKE 22829' ] #the bike that appears in India
single_bike1

In [ ]:
single_bike2 = df[df['name']== 'BIKE 22447' ] #the bike that spent days in Bonn
single_bike2

In [26]:
#single_bike3 = df_no_adr[df_no_adr['name']== 'BIKE 22385' ]
#single_bike3

In [27]:
no_duplicates = set(suspicious_distances) - set(suspicious_return_trips)

In [28]:
suspicious_indexes = suspicious_return_trips + list(no_duplicates)
suspicious_indexes

[1863,
 1864,
 9645,
 9646,
 9646,
 9647,
 44096,
 44097,
 23173,
 23176,
 13962,
 13964,
 23511,
 23512,
 36634,
 36635,
 39716,
 39717,
 55589,
 55590,
 7275,
 51948,
 51949,
 9712,
 9713,
 49139,
 49140,
 26549,
 26550,
 12857,
 12858,
 43582,
 43583]

In [29]:
#This is to be used only if we want to include the data about the bike that spent days out of Cologne in the list for droping
#out_of_cologne = set(single_bike2.index)
#list_for_dropping = set(suspicious_indexes) - out_of_cologne
#list_for_dropping = suspicious_indexes + list(list_for_dropping)

In [30]:
df_no = df.drop(suspicious_indexes, axis=0)
df_no.reset_index(inplace=True)
df_no.drop('index', axis=1, inplace=True)

In [31]:
df_no.head(10)

,lat_begin,lng_begin,lat_end,lng_end,ride_begin,ride_end,ride_time_minutes,distance,uid_begin,uid_end,standing_before_ride,address,name,available_bikes
0,50.921618,6.933237,50.913049,6.922490,2020-07-01 09:31:02.322428,2020-07-01 09:49:01.995455,18.0,1215.0,31804315,31819860,571.0,NaN,BIKE 22933,1093
1,50.913049,6.922490,50.910794,6.941967,2020-07-01 10:35:02.336121,2020-07-01 10:44:02.333799,9.0,1389.0,31819860,31821838,46.0,NaN,BIKE 22933,1159
2,50.910794,6.941967,50.931395,6.918226,2020-07-01 11:32:02.897704,2020-07-01 11:55:01.902627,23.0,2832.0,31821838,31824497,48.0,NaN,BIKE 22933,1161
3,50.931395,6.918226,50.929816,6.913806,2020-07-01 12:03:01.601807,2020-07-01 12:24:02.273590,21.0,356.0,31824497,31825694,8.0,NaN,BIKE 22933,1166
4,50.929816,6.913806,50.929823,6.913615,2020-07-01 17:59:02.213975,2020-07-01 18:05:02.088433,6.0,13.0,31825694,31843292,335.0,NaN,BIKE 22933,1160
5,50.929823,6.913615,50.923574,6.951995,2020-07-01 18:19:02.935191,2020-07-01 18:37:01.857777,18.0,2779.0,31843292,31845584,14.0,NaN,BIKE 22933,1162
6,50.923574,6.951995,50.933254,6.923639,2020-07-01 18:44:01.455040,2020-07-01 18:59:01.787044,15.0,2261.0,31845584,31847145,7.0,NaN,BIKE 22933,1172
7,50.933254,6.923639,50.927878,6.908989,2020-07-01 21:25:02.080594,2020-07-01 21:34:01.862246,9.0,1188.0,31847145,31856891,146.0,NaN,BIKE 22933,1170
8,50.927878,6.908989,50.928780,6.914153,2020-07-01 22:11:03.084599,2020-07-01 22:15:02.192482,4.0,376.0,31856891,31858979,37.0,NaN,BIKE 22933,1144
9,50.928780,6.914153,50.928696,6.913970,2020-07-02 19:14:02.079544,2020-07-02 19:16:02.319529,2.0,16.0,31858979,31901420,1259.0,NaN,BIKE 22933,1122


In [56]:
df_no.to_csv('clust_data_nextbike_2020-07.csv', sep=';', index=False) 

In [ ]:
df_no[df_no[""]]